In [1]:
import scipy as sp
import numpy as np
from sklearn import decomposition
from sklearn import cluster
import math

In [2]:
%run load_dino sulcus-new-5-3-134.out

In [3]:
np.shape(paths)
np.shape(paths)[0]
filtered_data = filter_paths(dinodata, .007)
indices = filtered_data[:,0]
paths = filtered_data[:,1:]
reshape_to_distances(paths)
np.shape(paths)

(18790, 795)

In [6]:
pca = decomposition.PCA(5)
m = pca.fit_transform(repaths)

In [ ]:
sum([  0.77505258,  0.11530088,  0.09391347,  0.00602776,  0.0037433
    ])

In [7]:
sum(pca.explained_variance_ratio_)

0.96898418991138435

In [4]:
def rebase_paths(paths, number_of_steps = 100):
    newpaths = np.empty((np.shape(paths)[0], 3 * number_of_steps))
    for i in range(np.shape(paths)[0]):
        if i % 1000 == 0:
            print(i)
        newpaths[i,:] = rebase_path(paths[i,:], number_of_steps)
    return newpaths

#Converts a path from list of points (at timesteps) to list of points (evenly distributed along the path)
def rebase_path(path, number_of_steps = 100):
    total_length = get_path_length(path)
    timesteps = int(math.floor(np.shape(paths)[1] / 3))
    newpath = np.zeros(number_of_steps * 3)
    current_timestep = 0
    current_distance = 0
    next_timestep_distance = 0
    last_timestep_distance = 0
    for i in range(number_of_steps):
        target_distance = i * 1.0 / number_of_steps * total_length
        next_timestep = current_timestep + 1
        #First we check to see which 2 points it's in between (current_timestep and next_timestep)
        while next_timestep_distance < target_distance:
            j = next_timestep
            if (j >= timesteps - 1):
                break
            next_timestep +=1
            last_timestep_distance = next_timestep_distance
            dx = path[3*j+3] - path[3*j]
            dy = path[3*j+4] - path[3*j+1]
            dz = path[3*j+5] - path[3*j+2]
            next_timestep_distance += math.sqrt(dx ** 2 + dy ** 2 + dz ** 2)
        current_timestep = next_timestep - 1
        #Now figure out how far between them it is
        j = current_timestep
        ax = path[3 * j]
        ay = path[3 * j + 1]
        az = path[3 * j + 2]
        bx = path[3 * j + 3]
        by = path[3 * j + 4]
        bz = path[3 * j + 5]
        try:
            lerp_ratio = (target_distance - last_timestep_distance) / (next_timestep_distance - last_timestep_distance)
        except:
            lerp_ratio = 0
        lerp_ratio = np.clip(lerp_ratio, 0, 1)
        newpath[3 * i] = (1 - lerp_ratio) * ax + lerp_ratio * bx
        newpath[3 * i + 1] = (1 - lerp_ratio) * ay + lerp_ratio * by
        newpath[3 * i + 2] = (1 - lerp_ratio) * az + lerp_ratio * bz
        #print(i,current_timestep,target_distance, lerp_ratio)
    return newpath

In [5]:
repaths = rebase_paths(paths,300)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000


In [8]:
target = 5000
p = target / np.shape(paths)[0]
print(p)
p = np.clip(p, 0, 1)
mask = np.random.choice([False, True], len(indices), p=[1-p, p])
ss_paths = paths[mask]
ss_indices = indices[mask]
ss_m = m[mask]
np.shape(ss_m)

0.2660989888238425


(5022, 5)

In [9]:
agglom = cluster.AgglomerativeClustering(n_clusters=50)
labels = agglom.fit_predict(ss_m)

In [ ]:
agglom.children_

In [ ]:
dir(agglom)

In [ ]:
np.histogram(labels, 50)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
plt.scatter(ss_m[:,0], ss_m[:,1], c=labels)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(ss_m[:,0], ss_m[:,1], ss_m[:,2], s=10, c=labels)

In [11]:
labelmap = {}
for i, label in enumerate(labels):
    if labelmap.get(label, []) == []:
        labelmap[label] = []
    labelmap[label].append(ss_indices[i])

In [ ]:
labelmap[0][0]

In [ ]:
#Method for selecting 1 point from each cluster
outlabels = [values[0] for label, values in labelmap.items()]
outstr = '\n'.join([str(int(v)) for v in outlabels])
#print(outstr)

In [ ]:
#method for printing out clusterings (cluster id per line)
outlabels = [(v, l) for l, vs in labelmap.items() for v in vs ]#for clusterings
outstr = '\n'.join([str(int(v)) + ' ' + str(int(l+1)) for v, l in outlabels])

In [12]:
#alternate method for printing clusterings (all members of a cluster on one line)
outlabels = [' '.join([str(int(i)) for i in items]) for _,items in labelmap.items()]
outstr = '\n'.join(outlabels)

In [13]:
with open('pca-rebase-zero-new-5-3-.7-50.clusters', 'w') as f:
    f.write(outstr)

In [15]:
1700-400+1000-800+17000+700-1300+16333-4000-13500-6000-10600

133